In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time

from tqdm import tqdm
import re

from urllib.parse import urljoin

In [4]:
# shangri_latibet

In [8]:
def extract_shangri_latibet_page_article_links(url,):
    """
    Extracts all article links from a given shangri_latibet webpage.

    This function scrapes the provided URL and extracts links to individual articles
    found on the page.

    Args:
    url (str): The URL of the shangri_latibet webpage containing article links.

    Returns:
        {
            "Links": List[],
            "Message": string,
            "Response": int,
            "source_url": string
        }
    Raises:
    requests.RequestException: If there's an error fetching the webpage.
    ValueError: If the expected HTML structure is not found on the page.
    """

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    final_response = {
        "Links": [],
        "Message": "Success",
        "Response": 200,
        "source_url": url
    }
    
    try:
        start_time = time.time()
        response = requests.get(url, headers=headers, timeout=(5, 60-5))
        response.raise_for_status()
        end_time = time.time()

        if end_time-start_time > 50:
            print(f"This ULR Took more then 50s: {url}")
        
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # # Getting all the links of articles 
        all_links = []
        all_link_article = soup.find("ul", id="breadcrumb")
        if all_link_article:
            article_block = all_link_article.find_all("a")
            if article_block:
                for each_link in article_block:
                    article_links = each_link.get("href")
                    if article_links:
                        all_links.append(article_links)
                        
        final_response["Links"] = all_links
        return final_response
     
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
    except requests.RequestException as e:
        # print(f"An error occurred while fetching the webpage: {e}")
        final_response["Message"] = f"An error occurred while fetching the webpage: {e}"
        final_response["Response"] = getattr(e.response, 'status_code', None)
        return final_response
    except ValueError as e:
        # print(f"An error occurred while parsing the webpage: {e}")
        final_response["Message"] = f"An error occurred while parsing the webpage: {e}"
        final_response["Response"] = 404
        # getattr(e.response, 'status_code', None)
        return final_response
    except Exception as e:
        # print(f"An unexpected error occurred: {e}")
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response


In [9]:

url = "http://www.shangri-latibet.cn/node_5112.html"
extract_shangri_latibet_page_article_links(url)

{'Links': ['http://www.shangri-latibet.cn/2024-09/19/content_430330.html',
  'http://www.shangri-latibet.cn/2024-09/19/content_430329.html',
  'http://www.shangri-latibet.cn/2024-09/18/content_430317.html',
  'http://www.shangri-latibet.cn/2024-09/12/content_430081.html',
  'http://www.shangri-latibet.cn/2024-09/12/content_430063.html',
  'http://www.shangri-latibet.cn/2024-09/12/content_430048.html',
  'http://www.shangri-latibet.cn/2024-09/12/content_430046.html',
  'http://www.shangri-latibet.cn/2024-09/11/content_430014.html',
  'http://www.shangri-latibet.cn/2024-09/11/content_430001.html',
  'http://www.shangri-latibet.cn/2024-09/11/content_430000.html',
  'http://www.shangri-latibet.cn/2024-09/10/content_429967.html',
  'http://www.shangri-latibet.cn/2024-09/06/content_429859.html',
  'http://www.shangri-latibet.cn/2024-09/05/content_429799.html',
  'http://www.shangri-latibet.cn/2024-09/04/content_429737.html',
  'http://www.shangri-latibet.cn/2024-09/04/content_429736.html',
 

In [8]:
# custom_url = 'https://ti.zangdiyg.com/category/index/id/61.html'
# print(custom_url.replace(".html", ""))

In [4]:
import requests
from bs4 import BeautifulSoup
import re
import time
import random
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from datetime import datetime, timedelta

def requests_retry_session(
    retries=3,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

def scrape_shangri_latibet_article_content(url, tags):
    headers = {
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.9,en-IN;q=0.8',
        'Cache-Control': 'max-age=0',
        'Connection': 'keep-alive',
        'Host': 'www.shangri-latibet.cn',
        'Referer': 'http://www.shangri-latibet.cn/node_5112_2.html',
        'Upgrade-Insecure-Requests': '1',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36 Edg/128.0.0.0'
    }
    
    # Add If-Modified-Since header
    headers['If-Modified-Since'] = (datetime.utcnow() + timedelta(days=18)).strftime('%a, %d %b %Y %H:%M:%S GMT')
    
    # Add If-None-Match header
    headers['If-None-Match'] = '"66dc7e9b-58da"'
    
    final_response = {
        "data": {
            'title': "",
            'body': {"Audio": "", "Text": []},
            'meta_data': {'URL': url, 'Author': "", 'Date': "", 'Tags': [tags]}
        },
        "Message": "Success",
        "Response": 200
    }
    
    try:
        # Add a random delay before making the request
        time.sleep(random.uniform(1, 3))
        
        # Make the request to the URL using the retry session
        session = requests_retry_session()
        response = session.get(url, headers=headers, allow_redirects=False)
        response.raise_for_status()
        
        # Check for redirect
        if response.is_redirect:
            final_response["Message"] = f"Redirected to: {response.headers['Location']}"
            final_response["Response"] = response.status_code
            return final_response
        
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        tags_body = soup.find("ul", class_="breadcrumb")
        tags = []
        if tags_body:
            tag_list = tags_body.find_all("a")
            if tag_list:
                for tag in tag_list:
                    each_tag = tag.get_text(strip=True)
                    tags.append(each_tag)
                final_response['data']['meta_data']["Tags"] = tags
        
        full_body = soup.find('div', class_="page")
        if full_body:
            # Extract title
            title = full_body.find('h1')
            if title:
                title_text = title.get_text(strip=True)
            else:
                title_text = ""
            final_response['data']["title"] = title_text
            
            metadata = full_body.find('p', class_="sx")
            # Extracting Meta Data
            try:
                if metadata:
                    all_text = metadata.get_text(strip=True)
                    if all_text:
                        text = all_text
                        print(text)
                        date_pattern = r'དུས་ཚོད།\s*(\d{4}-\d{2}-\d{2})'
                        author_source_pattern = r'(རྩོམ་པ་པོ།|རྩོམ་ཁུངས།)\s*(.+?)\s*(?:དུས་ཚོད།|རྩོམ་པ་པོ།|རྩོམ་ཁུངས།|$)'
                        # Extract date
                        date_match = re.search(date_pattern, text)
                        date = date_match.group(1) if date_match else ""
                        # Extract author and source
                        author_source = []
                        for match in re.finditer(author_source_pattern, text):
                            label = match.group(1)
                            content = match.group(2).strip()
                            author_source.append(f"{label} {content}")
                    
                        # Combine author and source
                        author_source_combined = " ".join(author_source)
                        final_response['data']['meta_data']["Date"] = date
                        final_response['data']['meta_data']["Author"] = author_source_combined
            except AttributeError:
                final_response['data']['meta_data']["Author"] = "Error fetching author"
                final_response['data']['meta_data']["Date"] = "Error fetching date"
            
        # Extract body content
        try:
            body = full_body.find("div")
            if body:
                paragraphs = body.find_all("p")
                if paragraphs:
                    final_response['data']['body']["Text"] = [para.get_text(strip=True) for para in paragraphs]
                else:
                    final_response['data']['body']["Text"] = [""]
        except AttributeError as e:
            final_response['data']['body']["Text"] = [f"Error fetching body content: {str(e)}"]
        
        return final_response
    except requests.Timeout:
        final_response["Message"] = "Request timed out"
        final_response["Response"] = 408  # Request Timeout
        return final_response
    except requests.RequestException as e:
        final_response["Message"] = f"An error occurred while fetching the article: {str(e)}"
        final_response["Response"] = getattr(e.response, 'status_code', 500)
        return final_response
    except Exception as e:
        final_response["Message"] = f"An unexpected error occurred: {e}"
        final_response["Response"] = 500
        return final_response

In [5]:
url = "http://www.shangri-latibet.cn/2024-08/29/content_429525.html"
# url = "https://www.tb1025.cn/lg/qijun/2017-09-29/2318.html"
url = "http://www.shangri-latibet.cn/2024-03/07/content_422525.html"
url = "http://www.shangri-latibet.cn/2024-07/01/content_427343.html"
scrape_shangri_latibet_article_content(url, tags="དཔྱད་གཏམ།")

རྩོམ་པ་པོ།      ཡིག་བསྒྱུར།   		    དུས་ཚོད།     &nbsp2024-07-01རྩོམ་ཁུངས།      西藏日报藏文媒体


{'data': {'title': 'གསར་དུ་བཀས་བཅད་པའི་ཐ་སྙད་འདི་དག་ཁྱེད་ཀྱིས་གཟིགས་བྱུང་ངམ།',
  'body': {'Audio': '',
   'Text': ['半脑山རི་མ་ལྡེབས།',
    '脑山རི་ལྡེབས།',
    '半舍饲ཕྱེད་ལག་གསོས།\u2002རྩྭ་ཆས་གཉིས་གསོ།',
    '帮虎吃食ངན་པའི་རྒྱབ་ལངས།',
    '报喜藏忧བཟང་སྒྲོན་ངན་སྐུང་།',
    '倍增效应ལྡབ་འཕར་ནུས་འབྲས།',
    '场景式བཟོས་ལྗོངས་རྣམ་པ།',
    '车补འཁོར་གསབ（གཞུང་འཁོར་གསབ་དངུལ）',
    '车改①རླངས་འཁོར་སྒྱུར་བཟོ། ②གཞུང་འཁོར་སྒྱུར་བཅོས།',
    '车载法庭འཁོར་ཐོག་ཁྲིམས་ར།',
    '创特提质གསར་གཏོད་སྤུས་འདེགས།',
    '错峰游འཚང་གཡོལ་ཡུལ་སྐོར།',
    '错峰出门འཚང་གཡོལ་ཕྱིར་སྐྱོད།',
    '充电站གློག་གསོག་ས་ཚིགས།',
    '大蓬车གུར་འཁོར་ཆེན་པོ།',
    '贷签对位བུན་གན་ཐོད་གཏུག',
    '电商达人གློག་ཚོང་བརྡར་པ།',
    '电信诈骗གློག་འཕྲིན་ཁ་བསླུས།',
    '顶雷གཞན་ཉེན་རང་འཁུར།',
    '顶流བཞུར་རྩེ（བཞུར་ཚད་རྩེ་གྲས）',
    '动力煤སྒུལ་ཤུགས་རྡོ་སོལ།',
    '督查问责ལྟ་བཤེར་འགན་འདེད།',
    '短视频བརྙན་ཐུང་།',
    '对抗性博奕ཁ་གཏད་འཐབ་འགྲན།',
    '多功能情节ནུས་མང་བྱུང་ཚུལ།',
    '多元共治སྣ་མང་མཉམ་བཅོས།\u2002རྒྱུ་མང་མཉམ་བཅོས།',
    '耳温枪རྣ་མདའ（རྣ་དྲོད་ལྟ་ཆས་ཤིག）',
    '仿生材料སྐྱེ་འགྲེའི་རྒྱུ་ཆ（སྐྱེ་དངོས་དཔེ་འགྲེའི

In [8]:
import re

def extract_info(text):
    # Remove extra spaces and replace &nbsp; with a space
    text = re.sub(' +', ' ', text)
    text = text.replace('&nbsp;', ' ')

    # Regular expressions for date and combined author/source
    date_pattern = r'དུས་ཚོད།\s*(\d{4}-\d{2}-\d{2})'
    author_source_pattern = r'(རྩོམ་པ་པོ།|རྩོམ་ཁུངས།)\s*(.+?)\s*(?:དུས་ཚོད།|རྩོམ་པ་པོ།|རྩོམ་ཁུངས།|$)'

    # Extract date
    date_match = re.search(date_pattern, text)
    if not date_match:
        # If date not found, try a more lenient pattern
        date_pattern = r'དུས་ཚོད།\s*(\d{4}-?\d{2}-?\d{2})'
        date_match = re.search(date_pattern, text)
    date = date_match.group(1) if date_match else "Date not found"

    # Extract author and source
    author_source = []
    for match in re.finditer(author_source_pattern, text):
        label = match.group(1)
        content = match.group(2).strip()
        author_source.append(f"{label} {content}")

    # Combine author and source
    author_source_combined = " ".join(author_source)

    return date, author_source_combined

# Test with the provided text
text = "རྩོམ་པ་པོ།      ཡིག་བསྒྱུར།   		    དུས་ཚོད།     &nbsp 2024-07-01 རྩོམ་ཁུངས།      西藏日报藏文媒体"

date, author_source = extract_info(text)
print(f"Date: {date}")
print(f"Author and Source: {author_source}")

Date: Date not found
Author and Source: རྩོམ་པ་པོ། ཡིག་བསྒྱུར། རྩོམ་ཁུངས། 西藏日报藏文媒体


In [10]:
import re

def extract_info(text):
    # Remove extra spaces
    text = re.sub(' +', ' ', text)

    # Regular expressions for date and combined author/source
    date_pattern = r'དུས་ཚོད།\s*(\d{4}-\d{2}-\d{2})'
    author_source_pattern = r'(རྩོམ་པ་པོ།|རྩོམ་ཁུངས།)\s*(.+?)\s*(?:དུས་ཚོད།|རྩོམ་པ་པོ།|རྩོམ་ཁུངས།|$)'

    # Extract date
    date_match = re.search(date_pattern, text)
    date = date_match.group(1) if date_match else "Date not found"

    # Extract author and source
    author_source = []
    for match in re.finditer(author_source_pattern, text):
        label = match.group(1)
        content = match.group(2).strip()
        author_source.append(f"{label} {content}")

    # Combine author and source
    author_source_combined = " ".join(author_source)

    return date, author_source_combined

# Test with the provided text
text = "རྩོམ་པ་པོ།     ཡིག་བསྒྱུར།       དུས་ཚོད།   2024-08-29   རྩོམ་ཁུངས།    སེམས་ཀྱི་ཉི་ཟླ་བོད་ཡིག་དྲ་བ།"
text = "རྩོམ་པ་པོ།      ཡིག་བསྒྱུར།   		    དུས་ཚོད།     &nbsp 2024-07-01 རྩོམ་ཁུངས།      西藏日报藏文媒体"

date, author_source = extract_info(text)
print(f"Date: {date}")
print(f"Author and Source: {author_source}")

Date: Date not found
Author and Source: རྩོམ་པ་པོ། ཡིག་བསྒྱུར། རྩོམ་ཁུངས། 西藏日报藏文媒体
